In [1]:
import json
from collections import Counter

import numpy as np
import pandas as pd
from sklearn.preprocessing import normalize

### [Question 1](#Question 1)

What are the top 3 and the bottom 3 states in terms of number of users?

In [6]:
with open('song.json') as jsonfile:
    data = json.load(jsonfile)


In [27]:
df = pd.DataFrame(data)
df.head()
len(df['user_id'].unique())
len(df)

4000

In [50]:
len(df.loc[df['user_state'] == 'New York']['user_id'].unique()
)

23

In [46]:
df.head()

,id,song_played,time_played,user_id,user_sign_up_date,user_state
0,GOQMMKSQQH,Hey Jude,2015-06-11 21:51:35,122,2015-05-16,Louisiana
1,HWKKBQKNWI,We Can Work It Out,2015-06-06 16:49:19,3,2015-05-01,Ohio
2,DKQSXVNJDH,Back In the U.S.S.R.,2015-06-14 02:11:29,35,2015-05-04,New Jersey
3,HLHRIDQTUW,P.s. I Love You,2015-06-08 12:26:10,126,2015-05-16,Illinois
4,SUKJCSBCYW,Sgt. Pepper's Lonely Hearts Club Band,2015-06-28 14:57:00,6,2015-05-01,New Jersey


In [79]:
df_state = df.groupby(['user_state','user_id']).size().reset_index(name='user_counts')

df_state.groupby('user_state').count().sort_values(by ='user_id', ascending = False).head()

,user_id,user_counts
user_state,,
New York,23,23
California,21,21
Texas,15,15
Pennsylvania,9,9
Ohio,9,9


<a id='Question 1'></a>
### Answer Q1
The top three states by count are-  New York, California, Texas
The bottom three states by count are - Nebraska, Arizona, Kansas

### [Question 2](#Question 2)

What are the top 3 and the bottom 3 states in terms of user engagement? You can choose how to mathematically deﬁne user engagement. What the CEO cares about here is in which states users are using the product a lot/very little

I think the user_engagment can be defined as how many times the user has listened to music

In [122]:
df['time_played_date'] = pd.to_datetime(df['time_played']).apply(lambda x: x.date())
df['user_sign_up_date'] = pd.to_datetime(df['user_sign_up_date']).apply(lambda x: x.date())
df['active'] = df['time_played_date'] - df['user_sign_up_date']
df['active'] = df['active'] / np.timedelta64(1, 'h')/24

0       26.0
1       36.0
2       41.0
3       23.0
4       58.0
5       41.0
6       33.0
7       17.0
8       26.0
9       27.0
10      44.0
11      14.0
12      21.0
13      40.0
14      42.0
15      34.0
16      46.0
17      20.0
18      34.0
19      31.0
20      26.0
21      35.0
22      32.0
23      31.0
24      47.0
25      42.0
26      49.0
27      14.0
28      28.0
29      18.0
        ... 
3970    30.0
3971    27.0
3972    27.0
3973    18.0
3974    31.0
3975    29.0
3976    37.0
3977    46.0
3978    39.0
3979    19.0
3980    31.0
3981    20.0
3982    25.0
3983    32.0
3984    45.0
3985    33.0
3986    28.0
3987    16.0
3988    16.0
3989    53.0
3990    17.0
3991    26.0
3992    24.0
3993    33.0
3994    48.0
3995    18.0
3996    37.0
3997    36.0
3998    15.0
3999    36.0
Name: active, Length: 4000, dtype: float64

In [117]:
type(df['active'][0])

pandas._libs.tslib.Timedelta

In [111]:
df_date=df.groupby(['user_state','user_id','time_played_date','active']).size().reset_index(name='date_counts')


In [113]:
df_date.groupby('user_state').mean()

,user_id,date_counts
user_state,,
Alabama,105.885246,1.704918
Alaska,155.444444,1.611111
Arizona,105.000000,1.466667
Arkansas,112.960000,1.360000
California,107.066421,1.568266
Colorado,175.081081,1.459459
Connecticut,127.000000,1.333333
Florida,88.009346,1.682243
Georgia,75.413793,1.551724
